### Bob möchte nun die Satoshis an ein spezielles von ihm erstelltes Skript binden. 
Das Skript ist der Satz: **"We make it visible"**. Auch dazu eignet sich die **P2SH** Transaktion.

In [ ]:
#bitcoind -regtest -daemon -fallbackfee=0.0000025 -> startet bitcoin in regtest mode
#bcr loadwallet alice (oder bcr createwallet alice)
#bcr loadwallet bob (oder bcr createwallet bob)
#bcr -rpcwallet=alice getnweaddress "" legacy -> gibt die addresse zur Vorbereitung von welcher dann die P2SH-Locking Transaktion ausgeht
#bcr -rpcwallet=alice sendtoaddress <address> <amount in btc>
#bcr -rpcwallet=alice generatetoaddress <amount of blocks> <address>
from io import BytesIO

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2pk_script, p2ms_script, p2sh_script
from src.ecdsa import PrivateKey
from src.crypto import hash160
from src.helper import string_to_hex

**1.** Zunächst muss Bob das von ihnen gewünschte Script erzeugen.

In [ ]:
bob_secret = "We make it visible"
bob_secret_hex = string_to_hex(bob_secret)
bob_original_script = Script([bytes.fromhex(bob_secret_hex), 0x87])
bob_original_script_hex = bob_original_script.serialize_script().hex()[2:]  
bob_original_script_hash160 = hash160(bytes.fromhex(bob_original_script_hex))

**1.1** In der ersten Transaktion werden von Alice die 10000 Satoshis an das P2SH Script von Bob versendet. Um das notwendige Transaktionsformular ausfüllen zu können, benötigt Alice folgende Informationen.

In [ ]:
#Transaktionsinputdaten
#bcr -rpcwallet=alice dumpprivkey <alice address> -> gibt den zu address gehörigen privaten Schlüssel in WIF
alice_private_key_wif_format = "cRT9RkTrJsXjhCu2t1U3eHoSQDtYnY6H9vkMMGrwQWG5o3kyKAS2"
alice_private_key = PrivateKey.convert_wif_format(alice_private_key_wif_format)
alice_private_key_int_format = alice_private_key.get_private_key_int()
alice_public_key = alice_private_key.get_public_key()

#bcr -rpcwallet=alice listunspent -> listet alle utxo's von alice auf
#bcr -rpcwallet=alice gettransaction <tx_id> -> gibt das "Überweisungsformular" der Transaktion mit der angegebenen ID zurück.
alice_previous_tx_id_to_spent = "cc9d1a1dc4bcf4c9c95b411d4ac47a2bd9ad8f864322d108797155f5fd922e10"
alice_previous_tx_index_to_spent = 1
alice_previous_script_pub_key_to_spent = "76a914b4b1f1bba135447439fab3d3eb2857169e912a6288ac"

#Transaktionsoutputdaten
alice_amount_to_spent = 10000

**1.2** Nun kann Alice die Lockingtransaktion erzeugen. Um die Satoshis an Bob's Skript zu senden, muss Alice im Output der Transaktion ein scriptPubKey der Form: OP_HASH160 + <script_hash> + OP_EQUAL generieren.

In [ ]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2sh_script(bob_original_script_hash160)
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2SH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

**1.3** Nun muss Alice die Lockingtransaktion noch signieren. D.h. sie muss nachweisen, dass sie aus der vorhergehenden Transaktion berechtigt ist die 10000 Satoshis auszugeben.

In [ ]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, [alice_private_key_int_format], alice_script_sig)

#bcr -rpcwallet=alice testmempoolaccept '["tx"]' -> prüft ob die Transaktion alle Checks für die Aufnhame in den Mempool besteht
#bcr -rpcwallet=alice decoderawtransaction <tx> -> decodiert die serilaisierte Transaktion ins "Überweisungsformular"
#bcr -rpcwallet=alice sendrawtransaction <tx> -> übermittelt schließlich die Transaktion im Netzwerk so dass sie von minern für einen block aus dem mempool aufgenommen werden kann

print(f"Alice signierte P2SH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

**2.** Nun möchte Bob die an sein Skript gebundenen Satoshis wieder ausgeben. In dem Fall muss er **keine** Signatur oder der gleichen erzeugen. Zum Beweis das er berechtigt ist die Funds auszugeben muss er im ScriptSig Feld des Inputs sein Secret und das Originalskript vorlegen.

In [ ]:
bob_previous_tx_id_to_spent = "2b85fc23dc16f8343fe2efe35d01a14451fc07e6154bbed9b3a19a669174b45c"
bob_previous_tx_index_to_spent = 0
bob_previous_script_pub_key_to_spent = "a914c7ccae441a968c81e8a49f92260473da4cd99d5587"

#Transaktionsoutputdaten
bob_amount_to_spent = 9000
#bob's reciever address is alice public key

**2.1** Nun kann Bob die Transaktion erstellen um die eben von Alice erhaltenen Satoshis weiter zu versenden.

In [ ]:
script_sig = Script([bytes.fromhex(bob_secret_hex), bytes.fromhex(bob_original_script_hex)])
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=script_sig)

script_pubkey = p2pk_script(alice_public_key.sec_format())
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

#bcr -rpcwallet=alice testmempoolaccept '["tx"]' -> prüft ob die Transaktion alle Checks für die Aufnhame in den Mempool besteht
#bcr -rpcwallet=alice decoderawtransaction <tx> -> decodiert die serilaisierte Transaktion ins "Überweisungsformular"
#bcr -rpcwallet=alice sendrawtransaction <tx> -> übermittelt schließlich die Transaktion im Netzwerk so dass sie von minern für einen block aus dem mempool aufgenommen werden kann

print(f"Bob 'signierte' P2SH Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")